In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguedashlineups
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import teamdashlineups
from nba_api.stats.endpoints import leaguegamelog
import statsmodels.api as sm
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import numpy as np
import time

In [2]:
def LineupData(season,per):
    tempdf1 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Base', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    tempdf2 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Advanced',
                                                  season = season, 
                                                  per_mode_detailed = per).get_data_frames()[0]
    time.sleep(1)
    tempdf3 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Four Factors', 
                                                  season = season,
                                                  per_mode_detailed = per).get_data_frames()[0]
    tempdf4 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Misc',
                                                  season = season, 
                                                  per_mode_detailed = per).get_data_frames()[0]
    time.sleep(1)
    tempdf5 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Scoring', 
                                                  season = season, 
                                                  per_mode_detailed = per).get_data_frames()[0]
    tempdf6 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Opponent',
                                                  season = season,
                                                  per_mode_detailed = per).get_data_frames()[0]
    time.sleep(1)
    
    tempdf = pd.merge(tempdf1,tempdf2, on= 'GROUP_ID', how='outer',suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf3, on= 'GROUP_ID', how='outer',suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf4, on= 'GROUP_ID', how='outer',suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf5, on= 'GROUP_ID', how='outer',suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf6, on= 'GROUP_ID', how='outer',suffixes = ('','_y'))
    
    tempdf = tempdf[tempdf.columns[~tempdf.columns.str.contains('RANK')]]
    tempdf = tempdf[tempdf.columns[~tempdf.columns.str.contains('_y')]]
    
    tempdf1 = tempdf[['GROUP_ID','PLUS_MINUS']].copy()
    
    print("done")
    return(tempdf1)

In [3]:
def LineupData1(season,per):
    tempdf = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Base', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    
    tempdf1 = tempdf[['GROUP_ID','GP','PLUS_MINUS']].copy()
    
    print("done")
    return(tempdf1)

In [4]:
def PlayerData(season,per):
    tempdf1 = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Base', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    tempdf2 = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Advanced', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    time.sleep(1)
    tempdf3 = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Usage', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    tempdf4 = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Misc', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    time.sleep(1)
    tempdf5 = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Scoring', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    tempdf6 = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Defense', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    
    tempdf = pd.merge(tempdf1,tempdf2, on='PLAYER_ID', how='outer', suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf3, on='PLAYER_ID', how='outer', suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf4, on='PLAYER_ID', how='outer', suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf5, on='PLAYER_ID', how='outer', suffixes = ('','_y'))
    tempdf = pd.merge(tempdf,tempdf6, on='PLAYER_ID', how='outer', suffixes = ('','_y'))
    
    tempdf = tempdf[tempdf.columns[~tempdf.columns.str.contains('RANK')]]
    tempdf = tempdf[tempdf.columns[~tempdf.columns.str.contains('_y')]]
    
    print('done')
    return(tempdf)

In [5]:
#Returns dictionary of scores
def nMax(n,playerids,attributes):
    teamscore = 0
    #Loop through players in lineup
    teamAttr = defaultdict(list)
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
            
        for attr in attributes:
            playerscore = playerstats.iloc[0][attr]
            teamAttr[attr].append(playerscore)

    rawAttr = defaultdict()
    for x in teamAttr:
        teamAttr[x].sort(reverse = True)
        teamAttr[x] = teamAttr[x][:n]
        rawAttr[x] = sum(teamAttr[x][:n])
    return(rawAttr)

In [6]:
def Train(df,attributes):
    for index, row in df.iterrows():
        playerids = row['GROUP_ID'].split("-")[1:-1]
        
        info = nMax(2,playerids,attributes)

        for key,value in info.items():
            df.at[index,key] = value
            
    return(df)

In [7]:
def ScoreCalc(df,attributes):
    df1 = Train(df,attributes)
    df1 = df1.loc[(df1['GP'] >= 10)]
    
    coeff = {}
    for attr in attributes:
        model = sm.OLS(df1['PLUS_MINUS'],df1[attr]).fit()
        val = model.tvalues[0] 
        coeff[attr] = val
    
    #calculate the scores
    df1['Score'] = (coeff[attributes[0]]*df1[attributes[0]] + coeff[attributes[1]]*df1[attributes[1]] + 
        coeff[attributes[2]]*df1[attributes[2]] + coeff[attributes[3]]*df1[attributes[3]] + 
        coeff[attributes[4]]*df1[attributes[4]] + coeff[attributes[5]]*df1[attributes[5]] + 
        coeff[attributes[6]]*df1[attributes[6]] + coeff[attributes[7]]*df1[attributes[7]])
    
#     df1['score'] = df1.apply(lambda x: coeff[attributes[0]]*df1[attributes[0]] + coeff[attributes[1]]*df1[attributes[1]] + 
#         coeff[attributes[2]]*df1[attributes[2]] + coeff[attributes[3]]*df1[attributes[3]] + 
#         coeff[attributes[4]]*df1[attributes[4]], axis=1)
    
    return(df1)
        

In [58]:
def GetLineups(season):
    nba_teams = teams.get_teams()

    teamLineups = defaultdict(str)
    for team in nba_teams:
        teamid = team['id']
        teamdf = teamdashlineups.TeamDashLineups(teamid, season = season,per_mode_detailed = 'Totals').get_data_frames()[1]
        #print(teamdf.head())
        teamdf.sort_values('MIN',ascending = False)
        teamLineups[team['abbreviation']] = teamdf.iat[0,1].split("-")[1:-1]
        #print("done")
        #print(teamLineups)
        time.sleep(2)
        
    return(teamLineups)

In [59]:
dfLineup = LineupData1('2017-18','Per100Possessions')
dfPlayer = PlayerData('2017-18','Per100Possessions')
print(len(dfLineup))
print(len(dfPlayer))
print(list(dfPlayer.columns))

done
done
2000
540
['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3', 'CFID', 'CFPARAMS', 'eOFF_RATING', 'OFF_RATING', 'sp_work_OFF_RATING', 'eDEF_RATING', 'DEF_RATING', 'sp_work_DEF_RATING', 'eNET_RATING', 'NET_RATING', 'sp_work_NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'ePACE', 'PACE', 'sp_work_PACE', 'PIE', 'FGM_PG', 'FGA_PG', 'PCT_FGM', 'PCT_FGA', 'PCT_FG3M', 'PCT_FG3A', 'PCT_FTM', 'PCT_FTA', 'PCT_OREB', 'PCT_DREB', 'PCT_REB', 'PCT_AST', 'PCT_TOV', 'PCT_STL', 'PCT_BLK', 'PCT_BLKA', 'PCT_PF', 'PCT_PFD', 'PCT_PTS', 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT', 'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT', 'PCT_FGA_2PT'

In [60]:
teamLineups = GetLineups('2017-18')
len(teamLineups)

30

In [81]:
attributes = ['AST','OREB','DREB','BLK','FG_PCT','FG3M','STL','TOV','OFF_RATING','DEF_RATING']
rand = nMax(2,['2585', '201142', '201939', '202691', '203110'],attributes)
print(rand)

defaultdict(None, {'AST': 18.9, 'OREB': 5.8, 'DREB': 20.4, 'BLK': 4.2, 'FG_PCT': 1.08, 'FG3M': 10.2, 'STL': 4.1, 'TOV': 8.5, 'OFF_RATING': 235.5, 'DEF_RATING': 215.3})


In [82]:
#Calculate score for each team
teamRank = defaultdict(int)
for key, value in teamLineups.items():
    vals = nMax(2,value,attributes)
    
    coeffs = {
        'AST': 2,
        'OREB': 10,
        'DREB': .5,
        'BLK': 1,
        'FG_PCT':50,
        'FG3M':3,
        'STL':1,
        'TOV':-5, 
        'OFF_RATING':1,
        'DEF_RATING':-.8
    }
    
    score = 0
    for key1,value1 in vals.items():
        score = score + (coeffs[key1]*value1)
    teamRank[key] = score
    
print(teamRank)

defaultdict(<class 'int'>, {'ATL': 179.59, 'BOS': 204.11999999999998, 'CLE': 175.49999999999997, 'NOP': 190.3, 'CHI': 164.13000000000002, 'DAL': 149.39, 'DEN': 195.88999999999996, 'GSW': 219.65999999999997, 'HOU': 242.90999999999994, 'LAC': 208.16, 'LAL': 153.54000000000002, 'MIA': 201.56, 'MIL': 186.52999999999997, 'MIN': 216.42999999999998, 'BKN': 200.89, 'NYK': 214.25, 'ORL': 177.38, 'IND': 189.44999999999996, 'PHI': 196.18, 'PHX': 187.01999999999998, 'POR': 192.14, 'SAC': 154.11, 'SAS': 211.92000000000002, 'OKC': 248.33, 'TOR': 219.87000000000006, 'UTA': 232.54000000000002, 'MEM': 168.83000000000004, 'WAS': 192.70999999999992, 'DET': 202.0, 'CHA': 207.48000000000002})


In [83]:
games = leaguegamelog.LeagueGameLog(season_all_time = '2017-18').get_data_frames()[0]
games.drop_duplicates('GAME_ID',inplace = True)
dummy = pd.get_dummies(games['WL'])
games = pd.concat([games,dummy],axis = 1)
print(len(games))
games.head()

1230


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,L,W
0,22017,1610612744,GSW,Golden State Warriors,0021700002,2017-10-17,GSW vs. HOU,L,240,43,...,34,5,9,17,25,121,-1,1,1,0
2,22017,1610612738,BOS,Boston Celtics,0021700001,2017-10-17,BOS @ CLE,L,240,36,...,24,11,4,12,24,99,-3,1,1,0
4,22017,1610612750,MIN,Minnesota Timberwolves,0021700011,2017-10-18,MIN @ SAS,L,240,37,...,23,7,4,13,16,99,-8,1,1,0
6,22017,1610612757,POR,Portland Trail Blazers,0021700012,2017-10-18,POR @ PHX,W,240,44,...,22,8,7,18,20,124,48,1,0,1
8,22017,1610612738,BOS,Boston Celtics,0021700007,2017-10-18,BOS vs. MIL,L,240,39,...,23,12,2,15,27,100,-8,1,1,0


In [86]:
#Add in Scores to games
for index, row in games.iterrows():
    home = row['MATCHUP'].split(' ')[0]
    away = row['MATCHUP'].split(' ')[2]
    games.at[index,'homeScore'] = teamRank[home]
    games.at[index,'awayScore'] = teamRank[away]
    games.at[index,'netScore'] = teamRank[home] - teamRank[away]
    
games.head(10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,L,W,homeScore,awayScore,netScore
0,22017,1610612744,GSW,Golden State Warriors,0021700002,2017-10-17,GSW vs. HOU,L,240,43,...,17,25,121,-1,1,1,0,219.66,242.91,-23.25
2,22017,1610612738,BOS,Boston Celtics,0021700001,2017-10-17,BOS @ CLE,L,240,36,...,12,24,99,-3,1,1,0,204.12,175.50,28.62
4,22017,1610612750,MIN,Minnesota Timberwolves,0021700011,2017-10-18,MIN @ SAS,L,240,37,...,13,16,99,-8,1,1,0,216.43,211.92,4.51
6,22017,1610612757,POR,Portland Trail Blazers,0021700012,2017-10-18,POR @ PHX,W,240,44,...,18,20,124,48,1,0,1,192.14,187.02,5.12
8,22017,1610612738,BOS,Boston Celtics,0021700007,2017-10-18,BOS vs. MIL,L,240,39,...,15,27,100,-8,1,1,0,204.12,186.53,17.59
10,22017,1610612766,CHA,Charlotte Hornets,0021700003,2017-10-18,CHA @ DET,L,240,29,...,17,15,90,-12,1,1,0,207.48,202.00,5.48
12,22017,1610612753,ORL,Orlando Magic,0021700005,2017-10-18,ORL vs. MIA,W,240,43,...,15,18,116,7,1,0,1,177.38,201.56,-24.18
14,22017,1610612754,IND,Indiana Pacers,0021700004,2017-10-18,IND vs. BKN,W,240,53,...,14,25,140,9,1,0,1,189.45,200.89,-11.44
16,22017,1610612743,DEN,Denver Nuggets,0021700010,2017-10-18,DEN @ UTA,L,240,36,...,21,18,96,-10,1,1,0,195.89,232.54,-36.65
18,22017,1610612737,ATL,Atlanta Hawks,0021700009,2017-10-18,ATL @ DAL,W,240,48,...,13,18,117,6,1,0,1,179.59,149.39,30.20


In [87]:
model = sm.OLS(games['PLUS_MINUS'],games['netScore']).fit()
model1 = sm.Logit(games['W'],games['netScore']).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 38.358483
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PLUS_MINUS   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     127.0
Date:                Thu, 21 Feb 2019   Prob (F-statistic):           4.24e-28
Time:                        04:41:31   Log-Likelihood:                -4914.2
No. Observations:                1230   AIC:                             9830.
Df Residuals:                    1229   BIC:                             9835.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
netScore       0.1210      0.011     11.269      0.000       0.100       0.142
==============================================================================
Omnibus:                       12.517   Durbin-Watson:                   1.885
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               14.553
Skew:                           0.168   Prob(JB):                     0.000691
Kurtosis:                       3.414   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
plt.figure(1)
plt.scatter(games['netScore'],games['PLUS_MINUS'])

In [69]:
from nba_api.stats.endpoints import commonteamroster
roster = commonteamroster.CommonTeamRoster(1610612744).get_data_frames()[0]

roster.head(20)

,TeamID,SEASON,LeagueID,PLAYER,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID
0,1610612744,2018,00,DeMarcus Cousins,0,C,6-11,270,"AUG 13, 1990",28.0,8,Kentucky,202326
1,1610612744,2018,00,Damion Lee,1,G,6-6,210,"OCT 21, 1992",26.0,1,Louisville,1627814
2,1610612744,2018,00,Jordan Bell,2,F,6-9,224,"JAN 07, 1995",24.0,1,Oregon,1628395
3,1610612744,2018,00,Quinn Cook,4,G,6-2,179,"MAR 23, 1993",25.0,2,Duke,1626188
4,1610612744,2018,00,Kevon Looney,5,C-F,6-9,220,"FEB 06, 1996",23.0,3,UCLA,1626172
5,1610612744,2018,00,Andre Iguodala,9,G-F,6-6,215,"JAN 28, 1984",35.0,14,Arizona,2738
6,1610612744,2018,00,Jacob Evans,10,G,6-6,210,"JUN 18, 1997",21.0,R,Cincinnati,1628980
7,1610612744,2018,00,Klay Thompson,11,G,6-7,215,"FEB 08, 1990",29.0,7,Washington State,202691
8,1610612744,2018,00,Damian Jones,15,C,7-0,245,"JUN 30, 1995",23.0,2,Vanderbilt,1627745
9,1610612744,2018,00,Jonas Jerebko,21,F,6-10,231,"MAR 02, 1987",31.0,8,"Kinna, Sweden",201973
